In [1]:
#!/usr/bin/env python
# coding: utf-8

if 'config' not in locals():
#     config = 'configs.config_WomenInEngineering_local'
    config = 'configs.config_ILLAE_local'    
#     config = 'configs.config_Juul_local'        

In [2]:
### initialization ###

import importlib
print('config file is set to {}'.format(config))

imp=importlib.import_module('_imports')
importlib.reload(imp)

util=importlib.import_module('_utils')
importlib.reload(util)

c=importlib.import_module(config)
importlib.reload(c)

get_ipython().run_line_magic('matplotlib', 'inline')

# param_setup(sys.argv[1:], c)
util.param_setup_ipython(globals(), c)

config file is set to configs.config_ILLAE_local


In [3]:
### functions for feature creation ###

def hashtags_counter(hashtags):
    if not isinstance(hashtags, str):
#         raise Exception('{} is not string. It is {}'.format(hashtags, type(hashtags)))
        print('{} is not string. It is {}'.format(hashtags, type(hashtags)))
        return [0, 0]
    hashtags_list=hashtags.split(',')
    sum_len = 0
    for h in hashtags_list:
        sum_len += len(h.strip())
    return [len(hashtags_list), sum_len/len(hashtags_list)]

def concat_cols(row):
    return ' '.join(map(str, row))

def concat_cols_df(df):
    if isinstance(df, imp.pd.DataFrame):
        if len(df.columns) == 1:
            return df
        return df.apply(concat_cols, axis=1)
    elif isinstance(df, imp.pd.Series):
        return df.apply(concat_cols)
    raise util.Failed('Input was not either dataframe or series!')

def preprocess_text(text, replacements):
#     replacements=list(replacements)
#     print(replacements)
    for replacement in replacements:
        try:
            text = imp.re.sub(*replacement, text)
        except TypeError as e:
            print('{} + text {}'.format(replacement, text))
            raise e
#     return tokenizer_tag(text)
    return text

def preprocess_text_series(series, replacements):
    return series.apply(preprocess_text, args=(replacements, ))

def calc_age_of_acct(acct_date):
    first_date=imp.date(2005, 1, 1)
    delta = acct_date.date() - first_date
    return delta.days

def calc_age_of_acct_series(series):
    return series.apply(calc_age_of_acct)

def char_len(text):
    return len(text)
    
def char_len_series(series):
    return series.apply(char_len)

def check_words(txt, words):
    txt=' '+txt.lower()+' '
    for word in words:
        word= ' ' + word + ' '
        if word in txt:
            return 1
    return 0

def check_words_series(words, series):
    return series.apply(check_words, args=(words,))

def anew(txt, emo_dict):
    words = imp.re.findall('[a-z]+', txt.lower())
    result={'valence':0,
            'dominance':0,
            'arousal':0
           }
    for key, emo in emo_dict.items():
        for word in words:
            if word in emo:
                result[key] += emo[word]
    return result

def anew_series(emo_dict, series):
    return series.apply(anew, args=(emo_dict,))

In [4]:
### preping the input file ###

util.mkdir('{}data/features/'.format(c.directory['save']))

dfs=dict()
dfs['input']=imp.pd.read_csv('{}data/{}_1_data_filtered.csv'.format(c.directory['save'],
                                                                    c.project_name))

folder /Users/habibkarbasian/Documents/_Courses/_Datasets/_Retweetability/_ILLAE/data/features/ has been created.


In [5]:
### creating static (meta) features and saving ###

for name, feature in c.features['static'].items():
    if feature['type']=='meta':
        cols=list([c.id_name])
        cols.extend(feature['cols'].keys())
        dfs[name]=dfs['input'][cols].copy()
        for col, col_type in feature['cols'].items():
            print('name = {}, type = {}'.format(col, col_type))

            if col_type == 'boolean':
                dfs[name][col] = (imp.pd.notna(dfs[name][col])).astype(float)
            elif col_type == 'age_of_account':
                dfs[name][col] = util.parallelize_df(dfs[name][col], util.str_date_to_date_df, c)
                dfs[name][col] = util.parallelize_df(dfs[name][col], calc_age_of_acct_series, c)
            elif col_type == 'length':
                dfs[name][col] = util.parallelize_df(dfs[name][col], char_len_series, c)
            elif col_type == 'lookup':
                dfs[name][col]=util.parallelize_df(dfs[name][col], check_words_series, c, 
                                                   params=(feature['words'], ))
            elif col_type == 'anew':
                ANEW = imp.pd.read_csv('./ANEW.csv')
                valence=dict(zip(ANEW['Word'], ANEW['valence']))
                dominance=dict(zip(ANEW['Word'], ANEW['dominance']))
                arousal=dict(zip(ANEW['Word'], ANEW['arousal']))

                emo_dict={'valence':valence,
                          'dominance':dominance,
                          'arousal':arousal
                         }
                results=util.parallelize_df(dfs[name][col], anew_series, c, params=(emo_dict,))
                dfs[name] = imp.pd.concat([dfs[name],
                                           imp.pd.DataFrame(list(results))
                                          ]
                                           , axis=1)
                dfs[name].drop([col], axis=1, inplace=True)
            elif col_type == 'hashtag':
                result=dfs[name][col].apply(hashtags_counter)
                result=list(zip(*result))
                dfs[name] = imp.pd.concat([dfs[name], 
                                           imp.pd.DataFrame(zip(*result), columns=['hashtags_count', 'hashtags_sum'])
                                          ]
                                          , axis=1) 
                dfs[name].drop([col], axis=1, inplace=True)

        dfs[name].to_csv('{}data/features/{}_feature_{}.csv'.format(c.directory['save'], 
                                                                   c.project_name,
                                                                   name
                                                                  ), index=False)
        imp.pprint('name of the dataframe = {}'.format(name))
        util.display_df(dfs[name].head())

name = friends_count, type = numerical
name = followers_count, type = numerical
name = favourites_count, type = numerical
name = statuses_count, type = numerical
'name of the dataframe = fffs'


,tweet_id,friends_count,followers_count,favourites_count,statuses_count
0,637005633548632064,34,946,78,1127
1,637007331574812673,196,253,117,672
2,637012950495031296,1287,13,2,3
3,637012998473670656,464,494,2341,8345
4,637014738216443905,416,206,16,85


name = listed_count, type = numerical
'name of the dataframe = listed'


,tweet_id,listed_count
0,637005633548632064,51
1,637007331574812673,11
2,637012950495031296,2
3,637012998473670656,2
4,637014738216443905,7


name = verified, type = numerical
'name of the dataframe = verified'


...

,tweet_id,hashtags_count,hashtags_sum
0,637005633548632064,2,14.5
1,637007331574812673,1,19.0
2,637012950495031296,1,19.0
3,637012998473670656,1,19.0
4,637014738216443905,2,13.5


name = in_reply_to_user_id, type = boolean
'name of the dataframe = reply'


,tweet_id,in_reply_to_user_id
0,637005633548632064,0.0
1,637007331574812673,0.0
2,637012950495031296,0.0
3,637012998473670656,1.0
4,637014738216443905,0.0


name = profile_created_at_date, type = age_of_account
parallel: 15 partitions with 15 cores for str_date_to_date_df
parallel: 15 partitions with 15 cores for calc_age_of_acct_series
'name of the dataframe = age'


,tweet_id,profile_created_at_date
0,637005633548632064,1586
1,637007331574812673,3175
2,637012950495031296,2316
3,637012998473670656,2465
4,637014738216443905,2904


In [6]:
### creating static (non-meta) features and saving ###

for  name, feature in c.features['static'].items():
    if feature['type']=='liwc':

        dfs[name]=imp.pd.read_csv('{}data/raw/{}'.format(c.directory['master'], 
                                                            feature['filename']))
        imp.pprint(dfs[name].shape)
        util.display_df(dfs[name].head())
    #     print(list(dfs[name]))

        index_id=list(dfs[name].iloc[0,:]).index(c.id_name)
        index_wc=list(dfs[name]).index('WC')
        dfs[name]=imp.pd.concat([dfs[name].iloc[1:, index_id], 
                                   dfs[name].iloc[1:, index_wc:]], 
                                  axis=1)
        col_names=list(dfs[name])
        col_names[0]=c.id_name
        dfs[name].columns=col_names
        dfs[name].to_csv('{}data/features/{}_feature_{}.csv'.format(c.directory['save'], 
                                                                    c.project_name,
                                                                    feature['type']), 
                         index=False)
        imp.pprint(dfs[name].shape)
        util.display_df(dfs[name].head())

(17355, 126)


/anaconda3/envs/RT/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (0,2,3,4,7,13,14,15,16,17,18,24) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Source (A),Source (B),Source (C),Source (D),Source (E),Source (F),Source (G),Source (H),Source (I),Source (J),Source (K),Source (L),Source (M),Source (N),Source (O),Source (P),Source (Q),Source (R),Source (S),Source (T),Source (U),Source (V),Source (W),Source (X),Source (Y),Source (Z),Source (AA),Source (AB),Source (AC),Source (AD),Source (AE),Source (AF),Source (AG),WC,Analytic,Clout,Authentic,Tone,WPS,Sixltr,Dic,function,pronoun,ppron,i,we,you,shehe,they,ipron,article,prep,auxverb,adverb,conj,negate,verb,adj,compare,interrog,number,quant,affect,posemo,negemo,anx,anger,sad,social,family,friend,female,male,cogproc,insight,cause,discrep,tentat,certain,differ,percept,see,hear,feel,bio,body,health,sexual,ingest,drives,affiliation,achieve,power,reward,risk,focuspast,focuspresent,focusfuture,relativ,motion,space,time,work,leisure,home,money,relig,death,informal,swear,netspeak,assent,nonflu,filler,AllPunc,Period,Comma,Colon,SemiC,QMark,Exclam,Dash,Quote,Apostro,Parenth,OtherP
0,in_reply_to_status_id,in_reply_to_screen_name,in_reply_to_user_id,retweet_count,favorite_count,tweet_created_at_date,tweet_created_at_hour,tweet_id,text,source,coordinates,name,screen_name,verified,friends_count,followers_count,favourites_count,statuses_count,listed_count,description,profile_created_at_date,profile_created_at_hour,location,language,user_id,profile_image_url_https,profile_background_image_url_https,profile_banner_url,external_url,user_mentions,hashtags,external_media_type,external_media_url,1,92.84,50.00,1.00,25.77,1.0,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00
1,NaN,NaN,NaN,0,0,Aug 27 2015,20:55:52,637005633548632064,#iLookLikeAnEngineer: Nuance employees share s...,"<a href=""http://socialchorus.com"" rel=""nofollo...",NaN,Nuance UK,NuanceUK,0,34,946,78,1127,51,"Nuance UK - the home of Dragon, PowerPDF and O...",May 06 2009,10:13:26,UK,en,38150806,https://pbs.twimg.com/profile_images/677196289...,https://abs.twimg.com/images/themes/theme9/bg.gif,https://pbs.twimg.com/profile_banners/38150806...,http://bit.ly/1PW3R9T,NaN,"iLookLikeAnEngineer, nuanceteam",NaN,NaN,21,92.84,92.33,1.00,95.81,21.0,23.81,61.90,38.10,9.52,0.00,0.0,0.00,0.0,0.0,0.0,9.52,4.76,19.05,9.52,4.76,0.00,0.0,9.52,4.76,4.76,4.76,0.0,0.0,4.76,4.76,0.0,0.0,0.0,0.0,14.29,0.0,0.0,4.76,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.52,4.76,0.00,4.76,0.00,0.0,0.00,14.29,0.0,4.76,0.00,4.76,0.00,4.76,0.0,0.0,0.0,0.0,0.0,4.76,0.0,4.76,0.0,0.0,0.0,33.33,4.76,0.0,4.76,0.0,0.0,0.0,0.0,0.0,4.76,0.0,19.05
2,NaN,NaN,NaN,1,0,Aug 27 2015,21:02:37,637007331574812673,We're so proud of our mentors. Earlier this mo...,"<a href=""http://www.hootsuite.com"" rel=""nofoll...",NaN,SparkProgramLA,SparkProgramLA,0,196,253,117,672,11,"Every day, we foster relationships between mid...",Sep 11 2013,17:42:45,"Los Angeles, CA",en,1854956568,https://pbs.twimg.com/profile_images/719566009...,https://pbs.twimg.com/profile_background_image...,https://pbs.twimg.com/profile_banners/18549565...,NaN,"1719657061, 135266722",ILookLikeAnEngineer,NaN,NaN,21,92.84,99.00,68.01,99.00,10.5,28.57,80.95,42.86,14.29,9.52,0.0,9.52,0.0,0.0,0.0,4.76,9.52,14.29,0.00,4.76,4.76,0.0,4.76,9.52,4.76,0.00,0.0,0.0,9.52,9.52,0.0,0.0,0.0,0.0,14.29,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,23.81,9.52,9.52,4.76,4.76,0.0,4.76,4.76,0.0,23.81,4.76,0.00,23.81,9.52,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,23.81,4.76,0.0,0.00,0.0,0.0,0.0,0.0,0.0,4.76,0.0,14.29
3,NaN,NaN,NaN,1,3,Aug 27 2015,21:24:57,637012950495031296,#Ilooklikeanengineer http://t.co/Zrz7y3Vt3X,"<a href=""http://www.apple.com"" rel=""nofollow"">...",NaN,kate,Szkolnik,0,1287,13,2

(17354, 94)


,tweet_id,WC,Analytic,Clout,Authentic,Tone,WPS,Sixltr,Dic,function,pronoun,ppron,i,we,you,shehe,they,ipron,article,prep,auxverb,adverb,conj,negate,verb,adj,compare,interrog,number,quant,affect,posemo,negemo,anx,anger,sad,social,family,friend,female,male,cogproc,insight,cause,discrep,tentat,certain,differ,percept,see,hear,feel,bio,body,health,sexual,ingest,drives,affiliation,achieve,power,reward,risk,focuspast,focuspresent,focusfuture,relativ,motion,space,time,work,leisure,home,money,relig,death,informal,swear,netspeak,assent,nonflu,filler,AllPunc,Period,Comma,Colon,SemiC,QMark,Exclam,Dash,Quote,Apostro,Parenth,OtherP
1,637005633548632064,21,92.84,92.33,1.00,95.81,21.0,23.81,61.90,38.10,9.52,0.00,0.0,0.00,0.0,0.0,0.0,9.52,4.76,19.05,9.52,4.76,0.00,0.0,9.52,4.76,4.76,4.76,0.0,0.0,4.76,4.76,0.0,0.0,0.0,0.0,14.29,0.0,0.0,4.76,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.52,4.76,0.00,4.76,0.00,0.0,0.00,14.29,0.0,4.76,0.00,4.76,0.00,4.76,0.00,0.0,0.0,0.0,0.0,4.76,0.0,4.76,0.0,0.0,0.0,33.33,4.76,0.0,4.76,0.0,0.0,0.0,0.0,0.0,4.76,0.0,19.05
2,637007331574812673,21,92.84,99.00,68.01,99.00,10.5,28.57,80.95,42.86,14.29,9.52,0.0,9.52,0.0,0.0,0.0,4.76,9.52,14.29,0.00,4.76,4.76,0.0,4.76,9.52,4.76,0.00,0.0,0.0,9.52,9.52,0.0,0.0,0.0,0.0,14.29,0.0,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,23.81,9.52,9.52,4.76,4.76,0.0,4.76,4.76,0.0,23.81,4.76,0.00,23.81,9.52,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,23.81,4.76,0.0,0.00,0.0,0.0,0.0,0.0,0.0,4.76,0.0,14.29
3,637012950495031296,6,92.84,50.00,1.00,25.77,6.0,33.33,16.67,0.00,0.00,0.00,0.0,0.00,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,16.67,0.0,16.67,0.0,0.0,0.0,66.67,16.67,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.0,50.00
4,637012998473670656,9,92.84,50.00,1.00,25.77,9.0,33.33,11.11,0.00,0.00,0.00,0.0,0.00,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,11.11,0.0,11.11,0.0,0.0,0.0,77.78,11.11,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.0,66.67
5,637014738216443905,19,92.84,70.08,50.35,25.77,9.5,36.84,47.37,10.53,0.00,0.00,0.0,0.00,0.0,0.0,0.0,0.00,5.26,0.00,0.00,5.26,0.00,0.0,5.26,5.26,5.26,0.00,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,5.26,0.0,0.0,0.00,0.0,5.26,0.0,5.26,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.53,5.26,0.00,5.26,0.00,0.0,10.53,5.26,0.0,21.05,10.53,0.00,10.53,5.26,5.26,0.0,0.0,0.0,0.0,5.26,0.0,5.26,0.0,0.0,0.0,36.84,10.53,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.0,26.32


In [7]:
### preparing data for dynamic features and saving ###

dynamics=dict()

for name, feature in c.features['dynamic'].items():
    print('{} - {}'.format(feature['type'], name))
    cols=list([c.id_name])
    cols.extend(feature['cols'])
    df_dynamic=dfs['input'][cols].copy()

#   the nodes column should be have comma as separator
    if feature['type'] not in dynamics:
        dynamics[feature['type']]=importlib.import_module('dynamic_features.{}'.format(feature['type']))
        importlib.reload(dynamics[feature['type']])
    df_dynamic = dynamics[feature['type']].prep_features(df_dynamic, feature, c)    
    df_dynamic.to_csv('{}data/features/{}_feature_{}({}).csv'.format(c.directory['save'], 
                                                                     c.project_name,
                                                                     feature['type'],
                                                                     name), 
                      index=False)

bow - bow_text
parallel: 15 partitions with 15 cores for concat_cols_df
parallel: 15 partitions with 15 cores for para_words_df
terms - terms_text
parallel: 15 partitions with 15 cores for concat_cols_df
parallel: 15 partitions with 15 cores for para_words_df
community - hashtags
lda - lda_hashtags
parallel: 15 partitions with 15 cores for concat_cols_df
parallel: 15 partitions with 15 cores for para_words_df
lda - full_text
parallel: 15 partitions with 15 cores for concat_cols_df
parallel: 15 partitions with 15 cores for para_words_df
